In [1]:
from matplotlib import pyplot as plt
import numpy as np
import torch
from torchvision import datasets
from torchvision import transforms


data_path = '../data/p1ch7/'

transformed_cifar10 = datasets.CIFAR10(data_path,train = True,download=False,
                                       transform=transforms.Compose([
                                           transforms.ToTensor(),
                                           transforms.Normalize((0.4915, 0.4823, 0.4468),
                                                                (0.2470, 0.2435, 0.2616))
                                       ]))

transformed_cifar10_val = datasets.CIFAR10(data_path,train = False,download=False,
                                       transform=transforms.Compose([
                                           transforms.ToTensor(),
                                           transforms.Normalize((0.4915, 0.4823, 0.4468),
                                                                (0.2470, 0.2435, 0.2616))
                                       ]))

label_map = {0:0,2:1}   # 把原来的标签映射过来

class_names = ['airplane','bird']

cifar2 = [(img,label_map[label]) for img,label in transformed_cifar10 
          if label in [0,2]] # 把 0 2 图像及对应标签取出，赋值给img和映射的标签
cifar2_val = [(img,label_map[label]) for img,label in transformed_cifar10_val 
              if label in [0,2]]

In [ ]:
import torch.nn as nn
import torch.optim as optim
model = nn.Sequential(
            nn.Linear(3072,512),
            nn.Tanh(),
            nn.Linear(512,2),
            nn.LogSoftmax(dim = 1))

learning_rate = 1e-1
optimizer = optim.SGD(model.parameters(),lr = learning_rate)
loss_fn = nn.NLLLoss()
n_epochs =100

# model.cuda()
for epoch in range(n_epochs):
    for img,label in cifar2:                               # 一个样本一个样本输入，没有batch
        out = model(img.view(-1).unsqueeze(0))
        loss = loss_fn(out,torch.tensor([label]))
        optimizer.zero_grad()                     # 将参数梯度置
        loss.backward()                           # 反向传播，计算梯度
        optimizer.step()                          # 更新参数，优化
    print ('Epoch: %d, Loss: %f' %(epoch,float(loss)))

Epoch: 0, Loss: 91.559662
Epoch: 1, Loss: 85.259102
Epoch: 2, Loss: 0.000000
Epoch: 3, Loss: 36.619743
Epoch: 4, Loss: 166.960663
Epoch: 5, Loss: 0.000713
Epoch: 6, Loss: 75.298431
Epoch: 7, Loss: 0.007719
Epoch: 8, Loss: 86.725845
Epoch: 9, Loss: 55.219326
Epoch: 10, Loss: 30.440813
Epoch: 11, Loss: 4.653501
Epoch: 12, Loss: 8.028452
Epoch: 13, Loss: 94.359947
Epoch: 14, Loss: 14.110658
Epoch: 15, Loss: 96.579865
Epoch: 16, Loss: 113.207542
Epoch: 17, Loss: 0.000000
Epoch: 18, Loss: 33.517952
Epoch: 19, Loss: 84.194016
Epoch: 20, Loss: 53.602997
Epoch: 21, Loss: 9.633520
Epoch: 22, Loss: 5.284300
Epoch: 23, Loss: 40.722492
Epoch: 24, Loss: 36.068230
Epoch: 25, Loss: 105.185059
Epoch: 26, Loss: 90.927505


In [ ]:
train_loader = torch.untils.data.DataLoader(cifar2,batch_size = 64,shuffle = True)

In [5]:
import torch.nn as nn
import torch.optim as optim

train_loader = torch.utils.data.DataLoader(cifar2,batch_size = 64,shuffle = True)
model = nn.Sequential(
            nn.Linear(3072,512),
            nn.Tanh(),
            nn.Linear(512,2),
            nn.LogSoftmax(dim = 1))

learning_rate = 1e-2
optimizer = optim.SGD(model.parameters(),lr = learning_rate)
loss_fn = nn.NLLLoss()
n_epochs =100

# model.cuda()
for epoch in range(n_epochs):
    for imgs,labels in train_loader:                           # imgs 维度[16, 3, 32, 32]，batchsize 为64
        batch_size = imgs.shape[0]
        out = model(imgs.view(batch_size,-1))
        loss = loss_fn(out,labels)
        optimizer.zero_grad()                     # 将参数梯度置
        loss.backward()                           # 反向传播，计算梯度
        optimizer.step()                          # 更新参数，优化
    print ('Epoch: %d, Loss: %f' %(epoch,float(loss)))

Epoch: 0, Loss: 0.336142
Epoch: 1, Loss: 0.510555
Epoch: 2, Loss: 0.499815
Epoch: 3, Loss: 0.563915
Epoch: 4, Loss: 0.243007
Epoch: 5, Loss: 0.383894
Epoch: 6, Loss: 0.380500
Epoch: 7, Loss: 0.365074
Epoch: 8, Loss: 0.415585
Epoch: 9, Loss: 0.443203
Epoch: 10, Loss: 0.435805
Epoch: 11, Loss: 0.230984
Epoch: 12, Loss: 0.336917
Epoch: 13, Loss: 0.464410
Epoch: 14, Loss: 0.170214
Epoch: 15, Loss: 0.200571
Epoch: 16, Loss: 0.194849
Epoch: 17, Loss: 0.602790
Epoch: 18, Loss: 0.345542
Epoch: 19, Loss: 0.508246
Epoch: 20, Loss: 0.277048
Epoch: 21, Loss: 0.589186
Epoch: 22, Loss: 0.168732
Epoch: 23, Loss: 0.188364
Epoch: 24, Loss: 0.222695
Epoch: 25, Loss: 0.332055
Epoch: 26, Loss: 0.552662
Epoch: 27, Loss: 0.337513
Epoch: 28, Loss: 0.437499
Epoch: 29, Loss: 0.203606
Epoch: 30, Loss: 0.179525
Epoch: 31, Loss: 0.209785
Epoch: 32, Loss: 0.210296
Epoch: 33, Loss: 0.173690
Epoch: 34, Loss: 0.241678
Epoch: 35, Loss: 0.247828
Epoch: 36, Loss: 0.193895
Epoch: 37, Loss: 0.103355
Epoch: 38, Loss: 0.103

In [28]:
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64,
shuffle=False)
correct =0
total = 0
with torch.no_grad():
    for imgs,labels in val_loader:
        batch_size = imgs.shape[0]
        outputs = model(imgs.view(batch_size,-1))
        _,predicted = torch.max(outputs,dim =1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
        
print('Accuracy: %f' %( correct / total))

Accuracy: 0.812000


In [22]:
imgs.shape

torch.Size([16, 3, 32, 32])

In [29]:
# 加深网络

model3 = nn.Sequential(
    nn.Linear(3072, 1024),
    nn.Tanh(),
    nn.Linear(1024, 512),
    nn.Tanh(),
    nn.Linear(512, 128),
    nn.Tanh(),
    nn.Linear(128, 2)
    )

loss_fn = nn.CrossEntropyLoss()

In [35]:
numel_list = [p.numel() for p in model.parameters() if p.requires_grad == True]

sum(numel_list), numel_list

(1574402, [1572864, 512, 1024, 2])